In [ ]:
# 1. Need to Get the Airport Location NY_Airport_Data [Done]

import csv
import pandas as pd
from lat_lon_parser import parse
import folium as fd
from IPython.display import HTML, display

airport_df = pd.read_csv(r'NY_Airport_Data.csv')
airport_df = airport_df[['ref_point_lat','ref_point_lon','objectid','fac_type','loc_id','region_code','state_name','county','fac_name']]
airport_df = airport_df[airport_df.fac_type == 'AIRPORT']
airport_df = airport_df.reset_index(drop=True)

def calculate_lat(row):
    val = parse(row['ref_point_lat'])
    return val
def calculate_lon(row):
    val = parse(row['ref_point_lon'])
    return val

airport_df['lat'] = (airport_df.apply(calculate_lat, axis=1)).round(2)
airport_df['lon'] = (airport_df.apply(calculate_lon, axis=1)).round(3)
airport_df = airport_df.reset_index(drop=True)
# v = parse("41-13-51.7000N")



In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import os

for i in range(104,367):

    netcdf_file_name = './data/windData/' + str(i) + '.nc4'
    csv_file_out = './data/windCSV/' + str(i) + '.csv'

    ds = xr.open_dataset(netcdf_file_name)
    df = ds.to_dataframe()
    print(df.describe())
    df.to_csv(csv_file_out)

    windspeed_df = pd.read_csv(csv_file_out)
    windspeed_df = pd.DataFrame(windspeed_df)

    windspeed_df = windspeed_df[['lat','lon','U50M','V50M','time']]
    windspeed_df['lat'] = windspeed_df['lat'].round(2)
    windspeed_df['lon'] = windspeed_df['lon'].round(2)
    windspeed_df['date'] = pd.to_datetime(windspeed_df['time']).dt.date

    csv_file_out_processed = './data/windCSV/' + str(i) + "_final.csv"
    windspeed_df.to_csv(csv_file_out_processed)
    os.remove(csv_file_out)


    # Compute the wind speed for the NewYork State on daily basis.....

    windspeed_df = pd.read_csv(csv_file_out_processed)
    windspeed_df = pd.DataFrame(windspeed_df)

    lon = windspeed_df['lon']
    lat = windspeed_df['lat']
    date = windspeed_df['date']

    # 2-meter eastward wind m/s
    U50M = windspeed_df['U50M']
    # 2-meter northward wind m/s
    V50M = windspeed_df['V50M']

    # Replace _FillValues with NaNs:df.replace(r'^\s*$', np.nan, regex=True)
    U2M_nans = U50M[:].replace(r'^\s*$', np.nan, regex=True)
    V2M_nans = V50M[:].replace(r'^\s*$', np.nan, regex=True)

    # Calculate wind speed:
    ws = np.sqrt(U2M_nans**2+V2M_nans**2)
    list_s = [lat, lon, date]

    new_df = pd.concat(list_s, axis=1)
    new_df['avg_windspeed'] = ws
    print(new_df.columns)
    new_df = pd.DataFrame(new_df.groupby(['lat','lon']).mean()).reset_index()
    new_df['date'] = date
    new_df.to_csv(csv_file_out_processed)



In [ ]:
def generate_blocks(csv_file_out):
    ws_df = pd.read_csv(csv_file_out)
    ws_df = pd.DataFrame(ws_df)

    _blocks = {
        'lower_left_lat': [],
        'lower_left_lon': [],
        'upper_right_lat': [],
        'upper_right_lon': [],
        'avg_speed': [],
        'date': [],

    }
    # retrieve lat lon list
    lat_list = sorted(set(ws_df['lat'].tolist()))
    lon_list = sorted(set(ws_df['lon'].tolist()))
    date_list = (set(ws_df['date'].tolist()))
    # for loop to generate all blocks
    # todo change 'avg windspeed' to wind speed
    speed = 0
    for x in range(len(lat_list) - 1):
        for y in range(len(lon_list) - 1):
            speed += ws_df[
                (ws_df['lat'] == lat_list[x]) &
                (ws_df['lon'] == lon_list[y])]['avg_windspeed'].tolist()[0]
            speed += ws_df[
                (ws_df['lat'] == lat_list[x + 1]) &
                (ws_df['lon'] == lon_list[y])]['avg_windspeed'].tolist()[0]
            speed += ws_df[
                (ws_df['lat'] == lat_list[x]) &
                (ws_df['lon'] == lon_list[y + 1])]['avg_windspeed'].tolist()[0]
            speed += ws_df[
                (ws_df['lat'] == lat_list[x + 1]) &
                (ws_df['lon'] == lon_list[y + 1])]['avg_windspeed'].tolist()[0]

            speed = speed / 4
            _blocks['lower_left_lat'].append(lat_list[x])
            _blocks['upper_right_lat'].append(lat_list[x + 1])
            _blocks['lower_left_lon'].append(lon_list[y])
            _blocks['upper_right_lon'].append(lon_list[y + 1])
            _blocks['avg_speed'].append(speed)
            _blocks['date'].append(ws_df.loc[x,'date'])
            speed = 0
        
        
    blocks = pd.DataFrame(_blocks)
    blocks.to_csv(csv_file_out)


for i in range(1,367):
    if(i==103): continue;
    csv_file_out = './data/windCSV/' + str(i) + '_final.csv'
    blocks = generate_blocks(csv_file_out)
    

In [ ]:
# Merge the data sets [Issues encountered]
def calculate_airport_avg_windspeed(airport_df, blocks):

    for row_id, row in airport_df.iterrows():
        lat_val = row.lat
        lon_val = row.lon
        filter1_a = lat_val >= blocks['lower_left_lat']
        filter1_b = lat_val <= blocks['upper_right_lat']
        filter2_a = lon_val >= blocks['lower_left_lon']
        filter2_b = lon_val <= blocks['upper_right_lon']
        
        calc_time = blocks[(filter1_a) & (filter1_b) & (filter2_a) & (filter2_b)].date.tolist()[0]
        calc_speed_temp = round(blocks[(filter1_a) & (filter1_b) & (filter2_a) & (filter2_b)].avg_speed.tolist()[0], 3)
        (airport_df.at[row_id, calc_time]) = calc_speed_temp
    
    return airport_df



airport_df_temp = airport_df.copy()
for i in range(1,367):
    if(i==103): continue;
    csv_file_out = './data/windCSV/' + str(i) + '_final.csv'
    blocks_ = pd.read_csv(csv_file_out)
    blocks_ = pd.DataFrame(blocks_)
    print(i)
    airport_df_temp = calculate_airport_avg_windspeed(airport_df_temp, blocks_)
    
final_dataset_with_speed_and_airport = airport_df_temp
final_dataset_with_speed_and_airport = final_dataset_with_speed_and_airport.reset_index(drop=True)
print(final_dataset_with_speed_and_airport)



In [ ]:
import datetime as dt

print(len(final_dataset_with_speed_and_airport.columns))
temp = final_dataset_with_speed_and_airport.copy()
lenght_val = len(temp.columns)
temp['Jan'] = 0; temp['Feb'] = 0; temp['Mar'] = 0;temp['Apr'] = 0;temp['May'] = 0;temp['June'] = 0;temp['July'] = 0;temp['Aug'] = 0;temp['Sep'] = 0;temp['Oct'] = 0;temp['Nov'] = 0;temp['Dec'] = 0
for cols in temp.columns[11:lenght_val]:
    val = pd.to_numeric(cols.split('-')[1]) 
    if val == 1:
        temp['Jan'] += temp[cols]
    elif val == 2:
        temp['Feb'] += temp[cols]
    elif val == 3:
        temp['Mar'] += temp[cols]
    elif val == 4:
        temp['Apr'] += temp[cols]
    elif val == 5:
        temp['May'] += temp[cols]
    elif val == 6:
        temp['June'] += temp[cols]
    elif val == 7:
        temp['July'] += temp[cols]
    elif val == 8:
        temp['Aug'] += temp[cols]
    elif val == 9:
        temp['Sep'] += temp[cols]
    elif val == 10:
        temp['Oct'] += temp[cols]
    elif val == 11:
        temp['Nov'] += temp[cols]
    elif val == 12:
        temp['Dec'] += temp[cols]

temp = temp.drop(temp.columns[11:lenght_val], axis = 1)
final_df_models = temp.copy()



In [ ]:
print(final_df_models.head())
final_df_models.to_csv('./data/windCSV/MonthlyMerged_final.csv')

Clustering Alogs: OPTICS

OPTICS clustering (where OPTICS is short for Ordering Points To Identify the Clustering Structure) is a modified version of DBSCAN described above.

We introduce a new algorithm for the purpose of cluster analysis which does not produce a clustering of a data set explicitly; but instead creates an augmented ordering of the database representing its density-based clustering structure. This cluster-ordering contains information which is equivalent to the density-based clusterings corresponding to a broad range of parameter settings.

— OPTICS: ordering points to identify the clustering structure, 1999.

The technique is described in the paper:

OPTICS: ordering points to identify the clustering structure, 1999.
It is implemented via the OPTICS class and the main configuration to tune is the “eps” and “min_samples” hyperparameters.

In [ ]:

import sklearn
print(sklearn.__version__)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.cluster import OPTICS, cluster_optics_dbscan
from sklearn.preprocessing import normalize, StandardScaler

In [ ]:
# Handling the missing values if any

airport_dataframe = final_df_models
county_list = final_df_models['county'].unique().tolist()

airport_dataframe['Category'] = airport_dataframe.county.astype("category").cat.codes
airport_dataframe_temp = airport_dataframe[['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec','Category']].copy()
temp.to_csv('./data/windData/'+ 'Final_DataSet.csv')

scaler = StandardScaler()
airport_dataframe_scaled = scaler.fit_transform(airport_dataframe_temp)
airport_normalized = normalize(airport_dataframe_scaled)
airport_normalized = pd.DataFrame(airport_normalized)
airport_normalized.columns = airport_dataframe_temp.columns
#print(airport_normalized.head())


# Building the OPTICS Clustering model
optics_model = OPTICS(min_samples = 50, xi = 0.05, min_cluster_size = 0.05)
  
# Training the model
optics_model.fit_predict(airport_normalized)

labels_050 = cluster_optics_dbscan(
    reachability=optics_model.reachability_,
    core_distances=optics_model.core_distances_,
    ordering=optics_model.ordering_,
    eps=0.5,
)
labels_200 = cluster_optics_dbscan(
    reachability=optics_model.reachability_,
    core_distances=optics_model.core_distances_,
    ordering=optics_model.ordering_,
    eps=2,
)

# Storing the cluster labels of each point
reachability = optics_model.reachability_[optics_model.ordering_]
labels = optics_model.labels_[optics_model.ordering_]
space = np.arange(len(airport_normalized))

print((labels))
print(reachability)

    

In [ ]:
# Defining the framework of the visualization
plt.figure(figsize =(20, 14))
G = gridspec.GridSpec(2, 3)
ax1 = plt.subplot(G[0, :])
ax2 = plt.subplot(G[1, 0])
ax3 = plt.subplot(G[1, 1])
ax4 = plt.subplot(G[1, 2])


# Reachability plot
colors = ["g.", "r.", "b.", "y.", "c."]
for klass, color in zip(range(0, 5), colors):
    Xk = space[labels == klass]
    Rk = reachability[labels == klass]
    ax1.plot(Xk, Rk, color, alpha=0.3)
ax1.plot(space[labels == -1], reachability[labels == -1], "k.", alpha=0.3)
ax1.plot(space, np.full_like(space, 2.0, dtype=float), "k-", alpha=0.5)
ax1.plot(space, np.full_like(space, 0.5, dtype=float), "k-.", alpha=0.5)
ax1.set_ylabel("Reachability (epsilon distance)")
ax1.set_title("Reachability Plot")


# Plotting the OPTICS Clustering
colors = ['c.', 'b.', 'r.', 'y.', 'g.']
for Class, colour in zip(range(0, 5), colors):
    Xk = airport_normalized[optics_model.labels_ == Class]
    ax2.plot(Xk.iloc[:, 0], Xk.iloc[:, 1], colour, alpha = 0.3)
      
ax2.plot(airport_normalized.iloc[optics_model.labels_ == -1, 0],
        airport_normalized.iloc[optics_model.labels_ == -1, 1],
       'k+', alpha = 0.1)
ax2.set_title('OPTICS Clustering')


# DBSCAN at 0.5
colors = ["g", "greenyellow", "olive", "r", "b", "c"]
print('# DBSCAN at 0.5')


for i in range(0,11):
    for klass, color in zip(range(0, 6), colors):
        Xk = airport_normalized[labels_050 == klass]
        ax3.plot(Xk[airport_normalized.columns[i]], Xk['Category'], color, alpha=0.3, marker=".")


# DBSCAN at 2.
colors = ["g.", "m.", "y.", "c."]
for i in range(0,11):
    for klass, color in zip(range(0, 4), colors):
        Xk = airport_normalized[labels_200 == klass]
        ax4.plot(Xk[airport_normalized.columns[i]], Xk['Category'], color, alpha=0.3)


for i in range(0,11):
    ax3.plot(airport_normalized.loc[labels_050 == -1, airport_normalized.columns[i]], airport_normalized.loc[labels_050 == -1, 'Category'], "k+", alpha=0.1)       
    ax4.plot(airport_normalized.loc[labels_200 == -1, airport_normalized.columns[i]], airport_normalized.loc[labels_200 == -1, 'Category'], "k+", alpha=0.1)

ax3.set_title("Clustering at 0.5 epsilon cut\nDBSCAN")
ax4.set_title("Clustering at 2.0 epsilon cut\nDBSCAN")

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt

https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a
Kmeans algorithm is an iterative algorithm that tries to partition the dataset into Kpre-defined distinct non-overlapping subgroups (clusters) where each data point belongs to only one group. It tries to make the intra-cluster data points as similar as possible while also keeping the clusters as different (far) as possible. It assigns data points to a cluster such that the sum of the squared distance between the data points and the cluster’s centroid (arithmetic mean of all the data points that belong to that cluster) is at the minimum. The less variation we have within clusters, the more homogeneous (similar) the data points are within the same cluster.

In [ ]:
kmeans_model_dataframe = final_df_models.copy()
kmeans_model_dataframe['Category'] = kmeans_model_dataframe.county.astype("category").cat.codes

kmeans_model_dataframe = kmeans_model_dataframe[['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec','Category']].copy()
kmeans_model_dataframe

scaler = StandardScaler()
kmeans_model_dataframe_norm = scaler.fit_transform(kmeans_model_dataframe)
kmeans_model_dataframe_norm = normalize(kmeans_model_dataframe_norm)
kmeans_model_dataframe_norm = pd.DataFrame(kmeans_model_dataframe_norm)
kmeans_model_dataframe_norm.columns = kmeans_model_dataframe_norm.columns
print(kmeans_model_dataframe_norm.head())


In [ ]:
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 20)
 
for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(kmeans_model_dataframe_norm)
    distortions.append(sum(np.min(cdist(kmeans_model_dataframe_norm, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / kmeans_model_dataframe_norm.shape[0])
    inertias.append(kmeanModel.inertia_)
    mapping1[k] = sum(np.min(cdist(kmeans_model_dataframe_norm, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / kmeans_model_dataframe_norm.shape[0]
    mapping2[k] = kmeanModel.inertia_


plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

plt.plot(K, inertias, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Inertia')
plt.title('The Elbow Method using Inertia')
plt.show()

In [ ]:
#Initialize the class object
kmeans = KMeans(n_clusters= 5)
 
#predict the labels of clusters.
label = kmeans.fit_predict(kmeans_model_dataframe)
 
print(label)

In [ ]:

#Getting the Centroids
centroids = kmeans.cluster_centers_
u_labels = np.unique(label)
 
#plotting the results:
plt.figure(figsize =(20, 7)) 
for i in u_labels:
    plt.scatter(kmeans_model_dataframe.loc[label == i , kmeans_model_dataframe.columns[11]] , kmeans_model_dataframe.loc[label == i , 'Category'] , label = i)

 
plt.legend()
plt.xlabel('Values of Avg Windspeed')
plt.ylabel('Cluster Labels')
plt.title('KMEANS Clustering for NY State for December Month')
plt.show()

plt.figure(figsize =(20, 7))
for i in u_labels:
    plt.scatter(kmeans_model_dataframe.loc[label == i , kmeans_model_dataframe.columns[10]] , kmeans_model_dataframe.loc[label == i , 'Category'] , label = i)

plt.legend()
plt.xlabel('Values of Avg Windspeed')
plt.ylabel('Cluster Labels')
plt.title('KMEANS Clustering for NY State for November Month')
plt.show()

plt.figure(figsize =(20, 7))
for i in u_labels:
    plt.scatter(kmeans_model_dataframe.loc[label == i , kmeans_model_dataframe.columns[0]] , kmeans_model_dataframe.loc[label == i , 'Category'] , label = i)

plt.legend()
plt.xlabel('Values of Avg Windspeed')
plt.ylabel('Cluster Labels')
plt.title('KMEANS Clustering for NY State for January Month')
plt.show()

Mean Shift Algorithm
Mean Shift is a hierarchical clustering algorithm. In contrast to supervised machine learning algorithms, clustering attempts to group data without having first been train on labeled data. Clustering is used in a wide variety of applications such as search engines, academic rankings and medicine. As opposed to K-Means, when using Mean Shift, you don’t need to know the number of categories (clusters) beforehand. The downside to Mean Shift is that it is computationally expensive — O(n²).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
meanshift_model_dataframe = final_df_models.copy()
meanshift_model_dataframe['Category'] = meanshift_model_dataframe.county.astype("category").cat.codes

meanshift_model_dataframe = meanshift_model_dataframe[['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec','Category']].copy()
print(meanshift_model_dataframe)

ms = MeanShift()
ms.fit(meanshift_model_dataframe)
cluster_centers = ms.cluster_centers_
print(cluster_centers)

In [ ]:
plt.figure(figsize =(20, 14))
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(meanshift_model_dataframe[meanshift_model_dataframe.columns[0]], meanshift_model_dataframe['Category'], marker='o', color='red')
ax.set_xlabel("Avergae Windspeed")
ax.set_ylabel("Labels")
ax.set_title("3D MeanShift Plot for January Month")
plt.show()

plt.figure(figsize =(20, 14))
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(meanshift_model_dataframe[meanshift_model_dataframe.columns[10]], meanshift_model_dataframe['Category'], marker='o', color='green')
ax.set_xlabel("Avergae Windspeed")
ax.set_ylabel("Labels")
ax.set_title("3D MeanShift Plot for November Month")
plt.show()

plt.figure(figsize =(20, 14))
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(meanshift_model_dataframe[meanshift_model_dataframe.columns[11]], meanshift_model_dataframe['Category'], marker='o', color='blue')
ax.set_xlabel("Avergae Windspeed")
ax.set_ylabel("Labels")
ax.set_title("3D MeanShift Plot for December Month")
plt.show()